In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn_pandas import DataFrameMapper
from sklearn.metrics import mean_squared_error, roc_auc_score
from scipy.integrate import trapz
import torch
import torchtuples as tt
from pycox.models import CoxCC
from pycox.evaluation import EvalSurv
from survival_evaluation import d_calibration, l1
import statistics
from pycox.models.cox_time import MLPVanillaCoxTime
from pycox.models import LogisticHazard, PMF, DeepHitSingle, CoxPH, MTLR, CoxTime
from sksurv.metrics import cumulative_dynamic_auc
import warnings
warnings.filterwarnings('ignore')

In [2]:
AUC_scores = []

# Load and process data
path = '../../data/NASA-Turbofan/Signature/train_new.csv'
path1 = '../../data/NASA-Turbofan/Signature/test_new.csv'

def load_data(path):
    D = pd.read_csv(path)
    # Configure column names
    x_cols = D.iloc[:, 4:].columns.tolist()  # Feature columns starting from the 5th column
    event_col = ['event']   # Survival event column
    time_col = ['time']     # Survival time column
    # Data cleaning and column selection
    D = D[x_cols + event_col + time_col]
    return D, x_cols

# Load training and testing datasets
d, x_cols = load_data(path)
d1, x_cols = load_data(path1)

# Columns to standardize
cols_standardize = x_cols.copy()
n_exp = 30  # Number of experiments

# Model evaluation metrics
CI = []         # Concordance Index
IBS = []        # Integrated Brier Score
L1_hinge = []   # L1 loss for hinge risk prediction
RMSE = []       # Root Mean Squared Error
AUC = []        # Area Under the Curve

In [3]:
# CoxPH model
L1_margin=[]
def CoxPH_():
    #df_train, df_val, df_test = train_test_split_with_val(d, 'event', frac_train=0.8, frac_val=0.05, frac_test=0.15, random_state=10)
    df_train=d
    df_val=d
    df_test=d1
    cols_standardize = [([col], StandardScaler()) for col in x_cols]
    cols_leave = [] 
    x_mapper = DataFrameMapper(cols_standardize + cols_leave)
    x_train = x_mapper.fit_transform(df_train).astype('float32')
    x_val = x_mapper.transform(df_val).astype('float32')
    x_test = x_mapper.transform(df_test).astype('float32')

    in_features = x_train.shape[1]

    get_target = lambda df: (df['time'].values, df['event'].values)
    y_train = get_target(df_train)
    y_val = get_target(df_val)
    y_test = get_target(df_test)
    durations_test, events_test = y_test

    out_features = 1
    num_nodes = [32, 32]
    batch_norm = True
    dropout = 0.1
    output_bias = False

    # Build Neural Network
    net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                                  dropout, output_bias=output_bias)

    model = CoxPH(net, tt.optim.Adam)

    batch_size = 256
    lr_finder = model.lr_finder(x_train, y_train, batch_size, tolerance=10)
    lr_finder.get_best_lr()
    model.optimizer.set_lr(0.01)

    epochs = 512
    callbacks = [tt.cb.EarlyStopping()]
    model.fit(x_train, y_train, batch_size, epochs, callbacks, val_data=(x_val, y_val), verbose=0, val_batch_size=batch_size)

    _ = model.compute_baseline_hazards()
    surv = model.predict_surv_df(x_test)  # Predict Survival Function

    # Add Time Series 'time' to the surv DataFrame
    surv_df = pd.DataFrame(surv)

    # Extract Row and Column Names
    surv_df.index.name = 'time'
    surv_df.columns.name = 'survival_function'

    # Save Row and Column Names to CSV
  


    survival_predictions = pd.Series(trapz(surv.values.T, surv.index), index=df_test.index)

    l1_hinge = l1(df_test.time, df_test.event, survival_predictions, l1_type='hinge')
    l1_margin = l1(df_test.time, df_test.event, survival_predictions, df_train.time, df_train.event, l1_type='margin')

    ev = EvalSurv(surv, durations_test, events_test, censor_surv='km')
    c_index = ev.concordance_td('antolini')
    time_grid = np.linspace(durations_test.min(), durations_test.max(), 10)
    brier = ev.integrated_brier_score(time_grid)

    quantiles = np.sort(df_test['time'].unique())

    # Calculate AUC
    labels_train = np.array([(e, t) for e, t in zip(df_train['event'], df_train['time'])], dtype=[('event', 'bool'), ('time', 'float')])
    labels_test = np.array([(e, t) for e, t in zip(df_test['event'], df_test['time'])], dtype=[('event', 'bool'), ('time', 'float')])

    time_grid_train = np.unique(df_train['time'])
    
    auc_scores = []
    for eval_time in quantiles: 
        try:
            interp_time_index = np.argmin(np.abs(eval_time - time_grid_train))
            surv_values_at_eval_time = surv.iloc[interp_time_index].values
            estimated_risks = 1 - surv_values_at_eval_time

            if np.min(estimated_risks) == np.max(estimated_risks): 
                continue  

            auc = cumulative_dynamic_auc(labels_train, labels_test, estimated_risks, times=[eval_time])[0][0]
            if not np.isnan(auc) and not np.isinf(auc):
                auc_scores.append(auc)
        except Exception as e:
            #print(f"AUC calculation failed: {e}, eval_time={eval_time}")
            pass

   

    return l1_hinge, l1_margin, c_index, brier, auc_scores, surv, df_test

def safe_stat(data):
    return round(statistics.mean(data), 3), round(statistics.stdev(data), 3) if len(data) > 1 else (0.0, 0.0)

def view_results(num_experiments):
    for i in range(num_experiments):
        l1_hinge, l1_margin, c_index, brier, auc_scores, surv, df_test = CoxPH_()
        L1_hinge.append(l1_hinge)
        L1_margin.append(l1_margin)
        CI.append(c_index)
        IBS.append(brier)
        AUC_scores.append(np.mean(auc_scores) if auc_scores else 0.5)  # set a default value of 0.5 (random level)
        auc_mean, auc_std = safe_stat(AUC_scores) 
       
    print('L1_hinge: {:.3f} ({:.3f})\n'.format(statistics.mean(L1_hinge), statistics.stdev(L1_hinge)))
    print('L1_margin: {:.3f} ({:.3f})\n'.format(statistics.mean(L1_margin), statistics.stdev(L1_margin)))
    print('CI: {:.3f} ({:.3f})\n'.format(statistics.mean(CI), statistics.stdev(CI)))
    print('IBS: {:.3f} ({:.3f})\n'.format(statistics.mean(IBS), statistics.stdev(IBS)))
    print(f'AUC: {auc_mean} ± {auc_std}')
    
    # d-calibration 
    d_cal = d_calibration(df_test.event, surv.iloc[6])
    print('d-calibration: \n')
    print('p_value: {:.4f}\n'.format(d_cal['p_value']))
    print('bin_proportions:')
    for i in d_cal['bin_proportions']:
        print(i)
    print('\n')
    print('censored_contributions:')
    for i in d_cal['censored_contributions']:
        print(i)
    print('\n')
    print('uncensored_contributions:')
    for i in d_cal['uncensored_contributions']:
        print(i)

    L1_hinge.clear()
    L1_margin.clear()
    CI.clear()
    IBS.clear()

view_results(30)

L1_hinge: 19.282 (6.613)

L1_margin: 44.421 (14.656)

CI: 0.493 (0.097)

IBS: 0.093 (0.028)

AUC: 0.503 ± 0.129
d-calibration: 

p_value: 0.0000

bin_proportions:
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.057307692307692164
0.4842307692307691


censored_contributions:
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.05730769230769234
0.057307692307692164
0.057307692307692164


uncensored_contributions:
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.4269230769230769
